In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from tensorflow import keras

In [2]:
cd /content/drive/MyDrive/comma_ai_dev/Probono-AI-Model

/content/drive/MyDrive/comma_ai_dev/Probono-AI-Model


In [75]:
from keras.models import load_model

#데이터 로드
df = pd.read_csv('/content/drive/MyDrive/comma_ai_dev/Probono-AI-Model/preprocessed_data/Gil_pop_data.csv',index_col=0, parse_dates=True)
df.index.freq = 'H' #index를 1시간 단위로 설정

# 모델 로드
model = load_model('/content/drive/MyDrive/comma_ai_dev/Probono-AI-Model/trained_models/gil_model.h5')
#parameter: n_input = 24, n_features = 1

In [76]:
recent_data = df[-24:] #가장 최근 24개의 데이터 이용

In [77]:
from sklearn.preprocessing import MinMaxScaler

 #데이터 scaling
scaler = MinMaxScaler()
scaler.fit(df)
scaled_dataset = scaler.transform(df[-24:])

In [78]:
#modeling 단계와 동일하게 변수설정
n_steps = 24 # 하루치 데이터
n_features = 1 #변수 개수
n_output = 24 # 출력 길이

In [79]:
pd.DataFrame(scaled_dataset).describe()

,0
count,24.000000
mean,0.564641
std,0.199399
min,0.308448
25%,0.367398
50%,0.590602
75%,0.733102
max,0.893209


In [80]:
# holding my predictions
predictions = []  # 예측 결과를 저장하기 위한 빈 배열

# 마지막 하루치 데이터부터 시작해서 하루치 예측
first_eval_batch = scaled_dataset[-n_steps:]

# RNN에 맞춰 timeseriesgenerator 출력 형식으로 형 변환 (batch 사이즈: 24) -> (n, 24)
current_batch = first_eval_batch.reshape(-1, n_steps)  # (1, 24)

# 예측할 범위 지정 (len_week 데이터 길이만큼 예측) -> 24의 크기를 가진 batch가 7개 필요
#for i in range(n_output):
current_pred = model.predict(current_batch)  # 한 배치를 통해 예측된 결과값 1개
# 예측값을 저장
predictions.append(current_pred)  # 예측한 결과를 하나씩 추가
# batch의 시작 포인트를 하나씩 뒤로 밀고, 새로운 예측값을 마지막에 저장하여 batch 업데이트
current_batch = np.append(current_batch[:, 1:], current_pred, axis=1)

1/1 [==============================] - 1s 574ms/step


In [81]:
#list 형식의 결과값을 numpy 형태로 변환
predictions = np.array(predictions).reshape(-1,1)
predictions = scaler.inverse_transform(predictions) #예측값 역정규화
predictions.shape

(1, 1)

In [91]:
from datetime import datetime, timedelta

# 예측 시작 시간 datetime index = 가장 마지막 datetime index + 1시간 후
last_datetime = df.iloc[-1:].index
pred_start_datetime = last_datetime + timedelta(hours=1)

# dataframe 형식으로 변경
df_predictions = pd.DataFrame(predictions, columns=['population'], index=pred_start_datetime)

df_predictions

,population
Datetime,
2023-09-01,58057.488281


In [92]:
recent_data = recent_data.append(df_predictions)
recent_data
#df_predictions.plot(figsize=(12,5))

<ipython-input-92-d1ddcf74bc00>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recent_data = recent_data.append(df_predictions)


,population
Datetime,
2023-08-31 00:00:00,58046.417300
2023-08-31 01:00:00,58072.746500
2023-08-31 02:00:00,58069.067800
2023-08-31 03:00:00,57838.022400
2023-08-31 04:00:00,57729.474500
2023-08-31 05:00:00,56807.213600
2023-08-31 06:00:00,55558.769400
2023-08-31 07:00:00,53790.162900
2023-08-31 08:00:00,52076.151200


In [ ]:
df.to_csv("Aug_pop_prediction.csv", index = True)